# 01.01 rootfinding : bisection

##### code, support for section text

In [1]:
if True: # settings for the people
  from itertools import count
  import matplotlib.animation
  import matplotlib.pyplot as plt
  import numpy as np
  import scipy as sp
  from tabulate import tabulate
  import textwrap


###### code, bisection

In [2]:
# bisection, basic

def bisect(f,ab,tol):

  a = ab[0]
  b = ab[1]
  while (b-a)/2 > tol:
    c = (a+b)/2
    fc = f(c)
    if fc == 0:
      break;
    fa = f(a)
    if fa*fc < 0:
      b = c
    else:
      a = c

  return c


In [3]:
# bisection, expanded for lecture

def bisect_expanded(f,ab,tol,all=False,workspace=False):

  if all:
    iterations = 0
  if workspace:
    ws = []
    i = 0

  a = ab[0]
  b = ab[1]
  while (b-a)/2 > tol:
    c = (a+b)/2
    fc = f(c)
    fs = 1 if fc > 0 else -1 if fc < 0 else 0 # bc all one dtype

    if fc == 0:
      if workspace:
        ws.append([i,a,fa,b,fb,c,fc,fs])
      break;
    if not 'fa' in locals(): # ie, local to function
      fa = f(a)
      if workspace:
        fb = f(b)
    if workspace:
      ws.append([i,a,fa,b,fb,c,fc,fs])
      i += 1

    if fa*fc < 0:
      b = c
      if workspace:
        fb = fc
    else:
      a = c
      fa = fc

    if all:
      iterations += 1
  if all:
    if workspace:
      return c,(a,b),iterations,ws
    else:
      return c,(a,b),iterations
  else:
    if workspace:
      return c,ws
    else:
      return c


###### code, images in order of appearance

In [4]:
# requires prior execution of bisect_expanded()

def ani_bisection(show="ani"):
  """
  show : "ani" animation, "data" (as table)
  """
  # prob-def
  f = lambda x: pow(x,3) + x - 1.
  ab = (0.,1.)
  tol = 1e-08

  # prof-def, runtime
  # calc
  root,ws = bisect_expanded(f,ab,tol,workspace=True)

  # cfg, runtime
  iterations = 3

  if show == "ani": # output, plot # True requires "ani" in a separate code cell
    # data, true, scatter
    h = 0.1
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    h_plot = h/10
    xs_plot = np.arange(ab[0]-h,ab[1]+h+h_plot,h_plot)
    ys_plot = f(xs_plot)

    # plot, cfg
    plt.close("all")
    fig,ax = plt.subplots()
    # plot, animation
    plt.ioff()
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    iani = count() # used in animate()

    # plot, actual
    dsize = 200 # lol if you try to show 5+ iterations BC WHY WOULD YOU EVER?!
    size = dsize*iterations + dsize/2
    ax.plot(xs_plot,ys_plot,zorder=1)
    ax.scatter(xs,ys,zorder=3)
    #ax.scatter(root,f(root),marker="*",s=size,zorder=2)
    ax.set_xlim(-0.1,1.1)
    ax.set_ylim(-1.1,1.1)
    ax2 = ax.twinx()
    ax.set_title("bisection: $x^3 + x - 1$, $x \in[0,1]$\n")
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    def animate(t):
      ax2.cla()
      offsets = {}
      k = next(iani)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=size,zorder=2)
      if k > 2:
        step = 2
        imax = k - 2 # first frame on ax2
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):
            if j == 0:
              xs_j = [ws[i,1],ws[i,3]]
              ys_j = [ws[i,2],ws[i,4]]
              acbs = ["a","b"]
            else:
              xs_j = [ws[i,5]]
              ys_j = [ws[i,6]]
              acbs = ["c"]
            ax2.scatter(xs_j,ys_j,color=color,marker=".",s=size-i*dsize,zorder=zorder+i)
            for acb,x,y in zip(acbs,xs_j,ys_j):
              label = "$" + acb+ "_{" + str(i) + "}$"
              if x in offsets:
                offsets[x] += 1
              else:
                offsets[x] = 1
              ax2.text(x-h_plot*2,y-h/2-h*offsets[x],label,color=color,fontweight="bold",zorder=zorder*2)

      ax2.set_xlim(-0.1,1.1)
      ax2.set_ylim(-1.1,1.1)

    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=8,interval=8)
    return ani

  else: # output, table
    if True: # use iterations
      print(tabulate(ws[0:iterations][:],headers=["i","a","f(a)","b","f(b)","c","f(c)","±"],intfmt="03d",floatfmt=".8f",tablefmt="github"))
      print(tabulate(ws[-iterations:][:],intfmt="03d",floatfmt=".8f",tablefmt="github"))
    else:
      print(tabulate(ws,headers=["i","a","f(a)","b","f(b)","c","f(c)","±"],intfmt="03d",floatfmt=".8f",tablefmt="github"))

#ani = ani_bisection()
#ani

###### code, examples

In [5]:
# example 01: basic, expanded, self-check

def eg_01():

  # problem
  f = lambda x: pow(x,3) + x - 1.
  ab = (0.,1.)
  tol = 1e-08

  # calc, basic
  if True:
    root = bisect(f,ab,tol)
    print(f"[basic algorithm] root: {root} at tolerance {tol}.\n")

  # calc, with details
  if True:
    root,ab_root,iters = bisect_expanded(f,ab,tol,all=True)
    s_answer = f"[algorithm expanded for details] \
      root {root} in final interval {ab_root} \
      after {iters} iterations at tolerance {tol}."
    print(textwrap.fill(" ".join(s_answer.split()),70),"\n")

  # calc, with scipy
  if True:
    root,rr = sp.optimize.bisect(f,ab[0],ab[1],xtol=tol,rtol=tol,full_output=True)
    iterations = rr.iterations # number of iterations # fyi
    print(f"[scipy] root: {root} took {iterations} iterations at tolerance {tol}.\n")


In [6]:
# example 02 modifies example 01

def eg_02():

  # problem
  f = lambda x: np.cos(x) - x
  ab = (0.,1.)
  tol = 0.5e-06

  # guess
  n = 6/np.log10(2)
  print(f"estimated steps: {n}.\n")

  # calc, with details
  if True:
    root,ab_root,iters = bisect_expanded(f,ab,tol,all=True)
    s_answer = f"[algorithm expanded for details] \
      root {root} in final interval {ab_root} \
      after {iters} iterations at tolerance {tol}."
    print(textwrap.fill(" ".join(s_answer.split()),70),"\n")

  # calc, with scipy
  if True:
    root,rr = sp.optimize.bisect(f,ab[0],ab[1],xtol=tol,rtol=tol,full_output=True)
    iterations = rr.iterations # number of iterations # fyi
    print(f"[scipy] root: {root} took {iterations} iterations at tolerance {tol}.\n")


## 1 bracketing a root

##### definition 01 root

function $f(x)$ has <b>root</b> at $x = r$ if $f(r) = 0$.

##### theorem 02: bolzano

<b>intermediate value theorem, <i>corollary 1</i>.</b> if a continuous function has values of opposite sign inside an interval, then it has a root in that interval.$^{[1]}$

In [ ]:
ani = ani_bisection()
ani

In [ ]:
ani_bisection(show="data")

|   i |          a |        f(a) |          b |       f(b) |          c |        f(c) |   ± |
|-----|------------|-------------|------------|------------|------------|-------------|-----|
| 000 | 0.00000000 | -1.00000000 | 1.00000000 | 1.00000000 | 0.50000000 | -0.37500000 | -01 |
| 001 | 0.50000000 | -0.37500000 | 1.00000000 | 1.00000000 | 0.75000000 |  0.17187500 | 001 |
| 002 | 0.50000000 | -0.37500000 | 0.75000000 | 0.17187500 | 0.62500000 | -0.13085938 | -01 |
|-----|------------|-------------|------------|------------|------------|-------------|-----|
| 023 | 0.68232775 | -0.00000014 | 0.68232787 | 0.00000015 | 0.68232781 |  0.00000001 | 001 |
| 024 | 0.68232775 | -0.00000014 | 0.68232781 | 0.00000001 | 0.68232778 | -0.00000006 | -01 |
| 025 | 0.68232778 | -0.00000006 | 0.68232781 | 0.00000001 | 0.68232779 | -0.00000003 | -01 |


##### algorithm <b>bisection</b>

```
# given x ∈[a,b] st f(a)⋅f(b) < 0

while (b-a)/2 > TOL
  c = (a+b)/2
  if f(c) = 0 then stop
  if f(a)⋅f(c) < 0 then
    b = c
  else
    a = c
  end
end

root_interval = [a,b]
root = (a+b)/2
```

##### example 01

find a root of function $f(x) = x^3 + x - 1$ on interval $[0,1]$ using bisection method.

In [ ]:
eg_01()

[basic algorithm] root: 0.6823277920484543 at tolerance 1e-08.

[algorithm expanded for details] root 0.6823277920484543 in final
interval (0.6823277920484543, 0.6823278069496155) after 26 iterations
at tolerance 1e-08. 

[scipy] root: 0.6823277920484543 took 26 iterations at tolerance 1e-08.



## 2 speed and accuracy

if continuous $f(x)$ and $x\in [a,b]$, then $[a_n,b_n]$ of length $\tfrac{b-a}{2^n}$ brackets the best solution after $n$ steps. ie, solution $r \approx x_c = \frac{a_n+b_n}{2}$ with

\begin{align}
  \text{error, bound:} &\qquad \Delta x < \epsilon \quad\Rightarrow\quad |x_c-x^*| < \frac{b-a}{2^{n+1}} \\
  \text{function evaluations:} &\qquad n+2.
\end{align}

assess the efficiency of bisection by accuracy gained with each function evaluation. ie, there is one function evaluation per step and each step halves uncertainty.

note: sure, the previously mentioned rounding error limit applies but the operations required per $f(x)$ are the same no matter how its usage within an approximation method. ie, its simpler to tally function calls.

##### definition 03


a solution is <b>correct within $p$ decimal places</b> if error is less than $0.5x10^{-p}$.

##### example 02

find root of $f(x)= cos\,x - x$ in interval $[0,1]$ within six correct places with bisection.

after $n$ steps,

\begin{align}
  |x_c-r| &< \frac{b-a}{2^{n+1}} \le \frac{1\times10^{-6}}{2} \\
  \\
  &\Downarrow \\
  \\
  \epsilon &= \frac{1-0}{2^{n+1}} \le 0.5\times10^{-6} \\
  \\
  &\Downarrow \\
  \\
  n &> \frac{6}{log_{10}2} \approx \frac{6}{0.301} \approx 19.9 \Rightarrow \text{ 20 steps required}.
\end{align}



In [ ]:
eg_02() # extends example 01

estimated steps: 19.931568569324174.

[algorithm expanded for details] root 0.7390851974487305 in final
interval (0.7390842437744141, 0.7390851974487305) after 20 iterations
at tolerance 5e-07. 

[scipy] root: 0.7390847206115723 took 21 iterations at tolerance 5e-07.



##### usw

error analysis helps set iteration limits, provides metrics when comparing efficiency (ie, accuracy gain per iteration).

## resources

- bisection [@scipy](https://docs.scipy.org/doc/scipy-1.13.1/reference/generated/scipy.optimize.bisect.html)

## references

[1] [bolzano, bernard](https://en.wikipedia.org/wiki/Bernard_Bolzano). <i>intermediate value theorem (IVT), corollary one</i>, [1817](https://en.wikipedia.org/wiki/Intermediate_value_theorem). also: bolzano-weierstrass, [some fifty years later](https://en.wikipedia.org/wiki/Bolzano%E2%80%93Weierstrass_theorem).